## Merge the scraped data (Excel Files)

In [4]:
import os
import pandas as pd
from zipfile import BadZipFile

# Set the directory where your Excel files are located
folder_path = '/Users/kittang/Desktop/Master Notes/SDSC 6014/Group Project Materials/Youtube Trending Project/DataSets/Final Raw Data'  # Replace with your folder path
output_file_name = 'Merged_Data.xlsx'  # Name of the merged output file

# List all Excel files in the folder
excel_files = [f for f in os.listdir(folder_path) if f.endswith(('.xlsx', '.xls'))]

# Initialize an empty list to hold dataframes
frames = []

# Loop through the files and read them into pandas dataframes
for file in excel_files:
    file_path = os.path.join(folder_path, file)
    try:
        # Determine the engine based on the file extension
        engine = 'openpyxl' if file.endswith('.xlsx') else 'xlrd'
        df = pd.read_excel(file_path, engine=engine)
        
        # Make sure 'Date' is a datetime type; replace 'Date' with your actual date column name
        df['Date'] = pd.to_datetime(df['Trending_Date'])
        
        # Sort the dataframe by the date column
        df.sort_values(by='Date', inplace=True)
        
        # Add a 'Trending Number' column that starts at 1 for each day
        df['Trending Number'] = df.groupby('Date').cumcount() + 1
        
        frames.append(df)
    except BadZipFile:
        print(f"Error reading {file}: File may be corrupt or not a .xlsx file.")
    except Exception as e:
        print(f"An error occurred while reading {file}: {e}")

# Concatenate all the dataframes into one merged dataframe
merged_df = pd.concat(frames, ignore_index=True)

# Write the merged dataframe to a new Excel file
merged_df.to_excel(output_file_name, index=False, engine='openpyxl')

print(f"Merged {len(frames)} out of {len(excel_files)} files into {output_file_name}")

Merged 51 out of 51 files into Merged_Data.xlsx


## Data Cleaning

### Removed column with same value in all / most rows

In [5]:
merged_df = merged_df.drop(columns=['HD','Disikes_Enabled', 'Dislike_Number','Fav_Number','Likes_Enabled','Disikes_Enabled'])
merged_df.describe()

,Trending_Date,Category_ID,View_Number,Like_Number,Comment_Number,Date,Trending Number
count,6715,6715.000000,6.715000e+03,6.693000e+03,6714.000000,6715,6715.000000
mean,2024-03-06 07:17:28.101265664,20.353239,1.580309e+07,4.849373e+05,7748.944296,2024-03-06 07:17:28.101265664,66.655249
min,2024-02-10 00:00:00,1.000000,3.844700e+04,0.000000e+00,0.000000,2024-02-10 00:00:00,1.000000
25%,2024-02-22 00:00:00,17.000000,3.520775e+05,9.194000e+03,181.000000,2024-02-22 00:00:00,33.000000
50%,2024-03-06 00:00:00,22.000000,1.091073e+06,3.248800e+04,512.000000,2024-03-06 00:00:00,66.000000
75%,2024-03-20 00:00:00,24.000000,1.574705e+07,4.021470e+05,3617.000000,2024-03-20 00:00:00,99.000000
max,2024-03-31 00:00:00,29.000000,2.384422e+08,8.864461e+06,300174.000000,2024-03-31 00:00:00,151.000000
std,NaN,5.933026,3.256622e+07,1.060866e+06,26616.044783,NaN,38.570991


### Removed rows with no like / comment number for an organized dataset

In [6]:
merged_df = merged_df[merged_df['Like_Number'].notna()]
merged_df = merged_df[merged_df['Comment_Number'].notna()]
merged_df.describe()

,Trending_Date,Category_ID,View_Number,Like_Number,Comment_Number,Date,Trending Number
count,6692,6692.000000,6.692000e+03,6.692000e+03,6692.000000,6692,6692.000000
mean,2024-03-06 06:47:33.317393920,20.359534,1.580945e+07,4.849872e+05,7765.678273,2024-03-06 06:47:33.317393920,66.605200
min,2024-02-10 00:00:00,1.000000,3.844700e+04,0.000000e+00,0.000000,2024-02-10 00:00:00,1.000000
25%,2024-02-22 00:00:00,17.000000,3.511815e+05,9.193000e+03,181.000000,2024-02-22 00:00:00,33.000000
50%,2024-03-06 00:00:00,22.000000,1.082094e+06,3.247650e+04,507.500000,2024-03-06 00:00:00,66.000000
75%,2024-03-20 00:00:00,24.000000,1.564320e+07,4.025708e+05,3626.500000,2024-03-20 00:00:00,99.000000
max,2024-03-31 00:00:00,29.000000,2.384422e+08,8.864461e+06,300174.000000,2024-03-31 00:00:00,151.000000
std,NaN,5.939829,3.262045e+07,1.060937e+06,26657.911881,NaN,38.522381


### Turn duration into seconds for EDA purpose

In [7]:
merged_df['DurationinSec'] = pd.to_datetime(merged_df['Duration'], format="%H:%M:%S", errors="coerce").fillna(pd.to_datetime(merged_df['Duration'], format="%M:%S", errors="coerce"))
merged_df['DurationinSec'] = (merged_df['DurationinSec'] - pd.Timestamp('1900-01-01')) // pd.Timedelta(seconds=1)

### Identified video with 60 sec or less than 60 sec as shorts

In [8]:
merged_df.loc[merged_df['DurationinSec'] <= 60, 'Shorts'] = 1
merged_df.loc[merged_df['DurationinSec'] > 60, 'Shorts'] = 0

### For rumor about trending video's title:

- Add tags in title
- Add number in Title
- Add emoji in title

In [9]:
merged_df.loc[merged_df['Video_Name'].str.contains("#"), 'TagsInTitle'] = 1
merged_df['TagsInTitle'] = merged_df['TagsInTitle'].fillna(0)

In [10]:
merged_df.loc[merged_df['Video_Name'].str.contains(r'\d'), 'NumInTitle'] = 1
merged_df['NumInTitle'] = merged_df['NumInTitle'].fillna(0)

In [11]:
merged_df.loc[merged_df['Video_Name'].str.contains(r'[\U00010000-\U0010ffff]'), 'EmojiInTitle'] = 1
merged_df['EmojiInTitle'] = merged_df['EmojiInTitle'].fillna(0)

### For checking whether it's a Chi or Eng Video

In [12]:
merged_df.loc[merged_df['Video_Name'].str.contains(r'[\u4e00-\u9fff\u3400-\u4dbf\U00020000-\U0002a6df\U0002a700-\U0002ebef\U00030000-\U000323af\ufa0e\ufa0f\ufa11\ufa13\ufa14\ufa1f\ufa21\ufa23\ufa24\ufa27\ufa28\ufa29\u3006\u3007][\ufe00-\ufe0f\U000e0100-\U000e01ef]?'), 'ChineseInTitle'] = 1
merged_df['ChineseInTitle'] = merged_df['ChineseInTitle'].fillna(0)

In [13]:
merged_df.loc[merged_df['Video_Name'].str.contains(r' [A-z][A-z]+'), 'EnglishInTitle'] = 1
merged_df['EnglishInTitle'] = merged_df['EnglishInTitle'].fillna(0)

In [14]:
merged_df.loc[(merged_df['ChineseInTitle'] == 1) & (merged_df['EnglishInTitle'] == 1), 'Eng&ChiInTitle'] = 1
merged_df['Eng&ChiInTitle'] = merged_df['Eng&ChiInTitle'].fillna(0)

In [15]:
merged_df.loc[(merged_df['ChineseInTitle'] == 0) & (merged_df['EnglishInTitle'] == 0), 'OtherLanguageInTitle'] = 1
merged_df['OtherLanguageInTitle'] = merged_df['OtherLanguageInTitle'].fillna(0)

### For checking whether it has Chi or Eng tags

In [21]:
merged_df.loc[merged_df['Tags'].str.contains(r'[\u4e00-\u9fff\u3400-\u4dbf\U00020000-\U0002a6df\U0002a700-\U0002ebef\U00030000-\U000323af\ufa0e\ufa0f\ufa11\ufa13\ufa14\ufa1f\ufa21\ufa23\ufa24\ufa27\ufa28\ufa29\u3006\u3007][\ufe00-\ufe0f\U000e0100-\U000e01ef]?',na=False), 'ChineseInTags'] = 1
merged_df['ChineseInTags'] = merged_df['ChineseInTags'].fillna(0)
merged_df.loc[merged_df['Tags'].str.contains(r',[A-z][A-z]+',na=False), 'EnglishInTags'] = 1
merged_df.loc[merged_df['Tags'].str.contains(r'[A-z][A-z]+,',na=False), 'EnglishInTags'] = 1
merged_df['EnglishInTags'] = merged_df['EnglishInTags'].fillna(0)
merged_df.loc[(merged_df['ChineseInTags'] == 1) & (merged_df['EnglishInTags'] == 1), 'Eng&ChiInTags'] = 1
merged_df['Eng&ChiInTags'] = merged_df['Eng&ChiInTags'].fillna(0)

In [22]:
print(merged_df)

     Trending_Date                   Channel_Name          Publish Date  \
0       2024-02-26            “獨生子的日常”YouTube官方頻道  2024-02-21T11:00:30Z   
1       2024-02-26  西川貴教 Official YouTube Channel  2024-02-09T09:00:24Z   
2       2024-02-26                        MrBeast  2024-02-10T17:00:00Z   
3       2024-02-26                          醒醒吧张律  2024-02-07T07:00:16Z   
4       2024-02-26              Hooper Highlights  2024-02-18T03:13:41Z   
...            ...                            ...                   ...   
6710    2024-03-24                     城寨 Singjai  2024-03-23T16:14:01Z   
6711    2024-03-24                     RFA 自由亞洲粵語  2024-03-22T23:00:06Z   
6712    2024-03-24                           FASH  2024-03-19T12:00:10Z   
6713    2024-03-24                          名侦探小宇  2024-03-10T10:00:27Z   
6714    2024-03-24                 陳蕾Panther Chan  2024-02-22T13:32:27Z   

                                             Video_Name  Category_ID  \
0                【獨生子的日常】小黄

In [23]:
merged_df.to_excel("cleanedDataset.xlsx", index=False)

In [24]:
df = pd.read_excel("cleanedDataset.xlsx")

In [26]:
df_exptags = df.copy()

# Expanding the 'Tags' into separate rows, similar to our previous approach
df_exptags = df_exptags.dropna(subset=['Tags']).assign(Expanded_Tags=df_exptags['Tags'].str.split(',')).explode('Expanded_Tags')

# Resetting index for cleanliness
df_exptags.reset_index(drop=True, inplace=True)

In [27]:
df_exptags = df_exptags.loc[df_exptags.groupby('Video_Name')['Trending_Date'].idxmax()]
df_exptags['Video_Name_Length'] = df_exptags['Video_Name'].str.len()

In [28]:
merged_df.to_excel("cleanedDatasetR.xlsx", index=False)